Which library we use for developing models will depend on the application. Many statistical problems can be solved by simpler techniques like ordinary least squares regression, while other problems may call for more advanced machine learning methods. 

In this chapter, we will review some featuers of pandas that may be helpful when we're crossing back and forth between data wrangling with pandas and model fitting and scoring. We will then look inoto short introductions to two popular modeling toolkits, stats-models and scikit-learn. 

<h3>Interfacing Between pandas and Model Code</h3>

A common workflow for model development is to use pandas for data loading and cleaning before switching over to a modeling library to build a model itself. An important part of the model development process is called <b>feature engineering</b> in machine learning. This can describe any data transformation or analytics that extract information from a raw dataset that may be useful in a modeling context. The data aggregation and GroupBy tools we have expored in this book are used often in a feature engineering context.

While details of 'good' feature engineering are out of scope for this book, we will explore some methods to make switching between data manipulation with pandas and modeling as painless as possible.

The point of contact between pandas and other analysis libraries is usually NumPy array. To turn a DataFrame into a NumPy array, use the <b>.values</b> property:

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.DataFrame({
    'x0': [1,2,3,4,5],
    'x1': [0.01,-0.01,0.25,-4.1,0.],
    'y': [-1.5,0.,3.6,1.3,-2.]
})

In [3]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [4]:
data.columns

Index(['x0', 'x1', 'y'], dtype='object')

In [5]:
data.values

array([[ 1.  ,  0.01, -1.5 ],
       [ 2.  , -0.01,  0.  ],
       [ 3.  ,  0.25,  3.6 ],
       [ 4.  , -4.1 ,  1.3 ],
       [ 5.  ,  0.  , -2.  ]])

In [6]:
type(data.values)

numpy.ndarray

To convert back to a DataFrame, as we may recall from earlier chapters, we can pass a two-dimensional ndarray with optional column nams:

In [7]:
df2 = pd.DataFrame(data.values, columns = ['one', 'two', 'three'])

In [8]:
df2

,one,two,three
0,1.0,0.01,-1.5
1,2.0,-0.01,0.0
2,3.0,0.25,3.6
3,4.0,-4.10,1.3
4,5.0,0.00,-2.0


The <b>.values</b> attribute is intended to be used when our data is homogeneous - for example, all numeric types. If we have heterogeneous data, the result will be an ndarray of Python objects:

In [9]:
df3 = data.copy()

In [10]:
df3['strings'] = ['a','b','c','d','e']

In [11]:
df3

,x0,x1,y,strings
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,c
3,4,-4.10,1.3,d
4,5,0.00,-2.0,e


In [12]:
df3.values

array([[1, 0.01, -1.5, 'a'],
       [2, -0.01, 0.0, 'b'],
       [3, 0.25, 3.6, 'c'],
       [4, -4.1, 1.3, 'd'],
       [5, 0.0, -2.0, 'e']], dtype=object)

For some models, we may only wish to use a subset of the columns. We recommend using <b>loc</b> indexing with values:

In [13]:
model_cols = ['x0', 'x1']

In [14]:
data.loc[:, model_cols].values

array([[ 1.  ,  0.01],
       [ 2.  , -0.01],
       [ 3.  ,  0.25],
       [ 4.  , -4.1 ],
       [ 5.  ,  0.  ]])

Some libraries have native support for pandas and do some of thsi work for us automatically; converting to NumPy from DataFrame and attachign model parameter names to the columns of output tables or Series. In other cases, we will have to perform this "metadata management" manually.

In previous chapter, we looked at pandas's Categorical type and the pandas.get_dummies function. Suppose we had a non-numeric column in our example dataset:

In [15]:
data['category'] = pd.Categorical(['a', 'b', 'a','a','b'],
                                 categories =['a','b'])

In [16]:
data

,x0,x1,y,category
0,1,0.01,-1.5,a
1,2,-0.01,0.0,b
2,3,0.25,3.6,a
3,4,-4.10,1.3,a
4,5,0.00,-2.0,b


If we wanted to replace the 'category' column with dummy variables, we create dummy variable, drop the 'category' column, and then join the result:

In [17]:
dummies = pd.get_dummies(data.category, prefix ='category')

In [18]:
dummies

,category_a,category_b
0,1,0
1,0,1
2,1,0
3,1,0
4,0,1


In [19]:
data_with_dummies = data.drop('category', axis = 1).join(dummies)

In [20]:
data_with_dummies

,x0,x1,y,category_a,category_b
0,1,0.01,-1.5,1,0
1,2,-0.01,0.0,0,1
2,3,0.25,3.6,1,0
3,4,-4.10,1.3,1,0
4,5,0.00,-2.0,0,1


There are some nuances to fitting certain statistical models with dummy variables. It may be simpler and less error-prone to use Patsy when we have more simple numeric columns.

<h3>Creating Model Description with Patsy</h3>

<b>Patsy</b> is a Python library for describing statistical models (especially linear models) with a small string-based "formula syntax", which is inspired by the formula syntax  used by R.

Patsy is well supported for specifying linear models in statsmodels, so we will focus on some of the main features to help us get up and runing.
Patsy's <b>formulas</b> are a special string syntax that looks like:
<pre>
y ~ x0 + x1
</pre>

The syntax a + b does not mean to add a to b, but rather that these are <b>terms</b> in the <b>design matrix</b> created for the model. The <b>patsy.dmatrices</b> functioni takes a formula string along with a dataset (which can be a DataFrame or a dict of arrays) and produces design matrices for a linear model:

In [21]:
data = pd.DataFrame({
    'x0': [1,2,3,4,5],
    'x1': [0.01, -0.01,0.25,-4.1,0.],
    'y': [-1.5,0.,3.6,1.3,-2.]
})

In [22]:
data

,x0,x1,y
0,1,0.01,-1.5
1,2,-0.01,0.0
2,3,0.25,3.6
3,4,-4.10,1.3
4,5,0.00,-2.0


In [23]:
import patsy

In [24]:
y, X = patsy.dmatrices('y ~ x0 + x1', data)

Now, we have:

In [25]:
y

DesignMatrix with shape (5, 1)
     y
  -1.5
   0.0
   3.6
   1.3
  -2.0
  Terms:
    'y' (column 0)

In [26]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0     x1
          1   1   0.01
          1   2  -0.01
          1   3   0.25
          1   4  -4.10
          1   5   0.00
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'x1' (column 2)

These Patsy <b>DesignMatrix</b> instances are NumPy ndarrays with additional metadata:


In [27]:
np.asarray(y)

array([[-1.5],
       [ 0. ],
       [ 3.6],
       [ 1.3],
       [-2. ]])

In [28]:
np.asarray(X)

array([[ 1.  ,  1.  ,  0.01],
       [ 1.  ,  2.  , -0.01],
       [ 1.  ,  3.  ,  0.25],
       [ 1.  ,  4.  , -4.1 ],
       [ 1.  ,  5.  ,  0.  ]])

We might wonder where the <b>Intercept</b> term came from . This is a convention for linear models like <b>Ordinary Least Squares (OLS) </b> regression. We can suppress the intercept by adding the term + 0 to the model:

In [29]:
patsy.dmatrices('y ~ x0 + x1 + 0', data)[1]

DesignMatrix with shape (5, 2)
  x0     x1
   1   0.01
   2  -0.01
   3   0.25
   4  -4.10
   5   0.00
  Terms:
    'x0' (column 0)
    'x1' (column 1)

The Patsy objects can be passed directly into algorithms like <b>numpy.linalg.lstq</b>, which performs an ordinary least squares regression:

In [30]:
coef, resid, _,_ = np.linalg.lstsq(X,y)

C:\Users\Predator\Anaconda3\envs\data_analysis\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: `rcond` parameter will change to the default of machine precision times ``max(M, N)`` where M and N are the input matrix dimensions.
To use the future default and silence this warning we advise to pass `rcond=None`, to keep using the old, explicitly pass `rcond=-1`.
  """Entry point for launching an IPython kernel.


The model metadata is retained in the <b>design_info</b> attribute, so we can reattach the model column names to the fitted coefficients to obtain a Series, for example:

In [31]:
coef

array([[ 0.31290976],
       [-0.07910564],
       [-0.26546384]])

In [32]:
coef = pd.Series(coef.squeeze(), index = X.design_info.column_names)

In [33]:
coef

Intercept    0.312910
x0          -0.079106
x1          -0.265464
dtype: float64

<h3>Data Transformations in Patsy Formulas</h3>

We can mix Python code into our Patsy fromulas; when evaluating the formula the library will try to find the functions we use in the enclosing scope:

In [37]:
y, X = patsy.dmatrices('y ~ x0 + np.log(np.abs(x1) + 1)', data)

In [38]:
X

DesignMatrix with shape (5, 3)
  Intercept  x0  np.log(np.abs(x1) + 1)
          1   1                 0.00995
          1   2                 0.00995
          1   3                 0.22314
          1   4                 1.62924
          1   5                 0.00000
  Terms:
    'Intercept' (column 0)
    'x0' (column 1)
    'np.log(np.abs(x1) + 1)' (column 2)

Some commonly used variable transformations include standardizing (to mean 0 and variance 1) and centering (subtracting the mean). Patsy has built-in functions for this purpose:

In [39]:
y, X = patsy.dmatrices('y ~ standardize(x0) + center(x1)', data)

In [40]:
X

DesignMatrix with shape (5, 3)
  Intercept  standardize(x0)  center(x1)
          1         -1.41421        0.78
          1         -0.70711        0.76
          1          0.00000        1.02
          1          0.70711       -3.33
          1          1.41421        0.77
  Terms:
    'Intercept' (column 0)
    'standardize(x0)' (column 1)
    'center(x1)' (column 2)

As part of a modeling process, we may fit a model on one dataset, then evaluate the model based on another. This might be a hold-out portion or new data that is observed later. When applying transformations like center and standardize, we should be careful when using the model to form predications based on new data. These are called <b>stateful</b> transformations, because we must use statistics like the mean or standard deviation of the original dataset when transforming a new dataset.

The <b>patsy.build_design_matrices</b> function can apply transformations to new out-of-sample data using the saved information from the original in-sample dataset:

In [41]:
new_data = pd.DataFrame({
    'x0': [6,7,8,9],
    'x1': [3.1,-0.5,0,2.3],
    'y': [1,2,3,4]
})

In [42]:
new_X = patsy.build_design_matrices([X.design_info], new_data)

In [43]:
new_X

[DesignMatrix with shape (4, 3)
   Intercept  standardize(x0)  center(x1)
           1          2.12132        3.87
           1          2.82843        0.27
           1          3.53553        0.77
           1          4.24264        3.07
   Terms:
     'Intercept' (column 0)
     'standardize(x0)' (column 1)
     'center(x1)' (column 2)]

Because the plus symbol (+) in the context of Ptasy formulas does not mean addition, when we want to add columns from a dataset by name, we must wrap them in the special I function

In [45]:
y, X = patsy.dmatrices('y ~ I(x0+x1)', data)

In [46]:
X

DesignMatrix with shape (5, 2)
  Intercept  I(x0 + x1)
          1        1.01
          1        1.99
          1        3.25
          1       -0.10
          1        5.00
  Terms:
    'Intercept' (column 0)
    'I(x0 + x1)' (column 1)

Patsy has several other built-in transforms in the patsy.builtins module. See the online documentation for more.

<h3>Categorical Data and Patsy</h3>

Non-numeric data can be transformed for a model design matrix in many different ways.

When we use non-numeric terms in a Patsy formula, they are converted to dummy variables by default. If there is an intercept, one of the levels will be left out to avoid collinearity:

In [47]:
data = pd.DataFrame({
    'key1': ['a', 'a', 'b','b','a', 'b','a','b'],
    'key2': [0,1,0,1,0,1,0,0],
    'v1':[1,2,3,4,5,6,7,8],
    'v2':[-1,0,2.5,-0.5,4.0,-1.2,0.2,-1.7]
    
})

In [48]:
y, X = patsy.dmatrices('v2 ~ key1', data)

In [49]:
X

DesignMatrix with shape (8, 2)
  Intercept  key1[T.b]
          1          0
          1          0
          1          1
          1          1
          1          0
          1          1
          1          0
          1          1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)

If we omit the intercept from the model, then columns for each category value will included in the model design matrix:

In [50]:
y, X = patsy.dmatrices('v2 ~ key1 + 0', data)

In [51]:
X

DesignMatrix with shape (8, 2)
  key1[a]  key1[b]
        1        0
        1        0
        0        1
        0        1
        1        0
        0        1
        1        0
        0        1
  Terms:
    'key1' (columns 0:2)

Numeric columns can be interpreted as categorial with the C functoin:

In [52]:
y, X = patsy.dmatrices('v2 ~ C(key2)', data)

In [53]:
X

DesignMatrix with shape (8, 2)
  Intercept  C(key2)[T.1]
          1             0
          1             1
          1             0
          1             1
          1             0
          1             1
          1             0
          1             0
  Terms:
    'Intercept' (column 0)
    'C(key2)' (column 1)

When we're using multiple categorial terms in a model, things can be more complicated, as we can include interaction terms of the form key1:key2, which can be used, for exmple, in analysis of variance (ANOVA) models:

In [54]:
data['key2'] = data['key2'].map({0:'zero', 1:'one'})

In [55]:
data

,key1,key2,v1,v2
0,a,zero,1,-1.0
1,a,one,2,0.0
2,b,zero,3,2.5
3,b,one,4,-0.5
4,a,zero,5,4.0
5,b,one,6,-1.2
6,a,zero,7,0.2
7,b,zero,8,-1.7


In [56]:
y, X = patsy.dmatrices('v2 ~ key1 + key2', data)

In [58]:
X

DesignMatrix with shape (8, 3)
  Intercept  key1[T.b]  key2[T.zero]
          1          0             1
          1          0             0
          1          1             1
          1          1             0
          1          0             1
          1          1             0
          1          0             1
          1          1             1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)

In [60]:
y, X = patsy.dmatrices('v2 ~ key1 + key2 + key1:key2', data)

In [61]:
X

DesignMatrix with shape (8, 4)
  Intercept  key1[T.b]  key2[T.zero]  key1[T.b]:key2[T.zero]
          1          0             1                       0
          1          0             0                       0
          1          1             1                       1
          1          1             0                       0
          1          0             1                       0
          1          1             0                       0
          1          0             1                       0
          1          1             1                       1
  Terms:
    'Intercept' (column 0)
    'key1' (column 1)
    'key2' (column 2)
    'key1:key2' (column 3)

Patsy provides for other ways to transform categorical data, including transformations for terms, with a particular ordering. 

<h3>Introduction to statsmodels</h3>

<b>statsmodels</b> is a Python library for fitting many kinds of statistical models, performing statistical tests, and data exploration and visualization. Statsmodels contains more "classical" frequentist statistical methods, while Bayesian methods and machine learning models are found in other libraries.

Some kinds of models found in statsmodels include:

   <ul>
       <li>Linear models, generalized linear models, and robust linear models
       <li>Linear mixed effects modesl</li>
       <li>Analysis of variacne (ANOVA) methods</li>
       <li>Time series processes and state space models</li>
       <li>Generalized method of moments</li>
   </ul>


<h3>Estimating Linear Models</h3>

There are several kinds of linear regression models in statsmodels, from the more basic (e.g ordinary least squares) to more complex (e.g, iteratively reweighted least squares).

Linear models in statsmodels have two different main interfaces: array-based and formula-based. These are accessed through these API module imports:

In [63]:
import statsmodels.api as sm
import statsmodels.formula.api as smf

To show how to use these, we generate a linear model for some random data:

In [64]:
def dnorm(mean, variance, size = 1):
    if isinstance(size, int):
        size = size,
    return mean + np.sqrt(variance) * np.random.randn(*size)

In [65]:
np.random.seed(12345)

In [66]:
N = 100

In [67]:
X = np.c_[dnorm(0,0.4,size=N),
         dnorm(0,0.6,size=N),
         dnorm(0,0.2,size=N)]

In [68]:
eps = dnorm(0,0.1,size=N)

In [70]:
beta = [0.1,0.3,0.5]

In [71]:
y = np.dot(X, beta) + eps

Here ,we wrote down the "true" model with known parameters beta. In this case, dnorm is a helper function for generating normally distributed data with a particular mean and variance. So now we have;

In [72]:
X[:5]

array([[-0.12946849, -1.21275292,  0.50422488],
       [ 0.30291036, -0.43574176, -0.25417986],
       [-0.32852189, -0.02530153,  0.13835097],
       [-0.35147471, -0.71960511, -0.25821463],
       [ 1.2432688 , -0.37379916, -0.52262905]])

In [73]:
y[:5]

array([ 0.42786349, -0.67348041, -0.09087764, -0.48949442, -0.12894109])

A linear model is generally fitted with an intercept term as we saw before with Patsy. The <b>sm.add_constant</b> function can add an intercept column to an existing matrix:

In [74]:
X_model = sm.add_constant(X)

In [75]:
X_model[:5]

array([[ 1.        , -0.12946849, -1.21275292,  0.50422488],
       [ 1.        ,  0.30291036, -0.43574176, -0.25417986],
       [ 1.        , -0.32852189, -0.02530153,  0.13835097],
       [ 1.        , -0.35147471, -0.71960511, -0.25821463],
       [ 1.        ,  1.2432688 , -0.37379916, -0.52262905]])

The sm.OLS class can fit an ordinary least square linear regression:

In [76]:
model = sm.OLS(y, X)

The model's fit method returns a regression results object containing estimated model parameters and other diagnostics:

In [77]:
results = model.fit()

In [78]:
results.params

array([0.17826108, 0.22303962, 0.50095093])

The summary method on results can print a model detailing diagnostic ooutput of the model:

In [79]:
print(results.summary())

                                 OLS Regression Results                                
Dep. Variable:                      y   R-squared (uncentered):                   0.430
Model:                            OLS   Adj. R-squared (uncentered):              0.413
Method:                 Least Squares   F-statistic:                              24.42
Date:                Tue, 26 Jan 2021   Prob (F-statistic):                    7.44e-12
Time:                        14:37:20   Log-Likelihood:                         -34.305
No. Observations:                 100   AIC:                                      74.61
Df Residuals:                      97   BIC:                                      82.42
Df Model:                           3                                                  
Covariance Type:            nonrobust                                                  
                 coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------

The parameter names here have been given the generic names x1, x2, and so on. Suppose instead that all of the model parameters are in a DataFrame:

In [80]:
data = pd.DataFrame(X, columns=['col0', 'col1', 'col2'])

In [81]:
data['y'] = y

In [82]:
data[:5]

,col0,col1,col2,y
0,-0.129468,-1.212753,0.504225,0.427863
1,0.302910,-0.435742,-0.254180,-0.673480
2,-0.328522,-0.025302,0.138351,-0.090878
3,-0.351475,-0.719605,-0.258215,-0.489494
4,1.243269,-0.373799,-0.522629,-0.128941


Now we can use the statsmodels formula API and Patsy formula strings:


In [83]:
results = smf.ols('y ~ col0 + col1 + col2', data=data).fit()

In [84]:
results.params

Intercept    0.033559
col0         0.176149
col1         0.224826
col2         0.514808
dtype: float64

In [85]:
results.tvalues

Intercept    0.952188
col0         3.319754
col1         4.850730
col2         6.303971
dtype: float64

Observe how statsmodels has returned results as Series with the DataFrame column names attached. We also do not need to use <b>add_constant</b> when using formulas and pandas objects.

Given, new out-of-sample data, we can compute predicted values given the estimated model parameters:

In [86]:
results.predict(data[:5])

0   -0.002327
1   -0.141904
2    0.041226
3   -0.323070
4   -0.100535
dtype: float64

There are many additional tools for analysis, diagnostics, and visualization of linear model results in statsmodels that we can explore. There are also other kinds of linear models beyond ordinary least squares.

<h3>Estimating Time Series Processes</h3>

Another class of models in statsmodels are for time series analysis. Among these are autoregressive processes, Kalman filtering and other state space modles, and multivariate autoregressive models.

Let's simulate some time series data with an autoregressive structure and noise:

In [87]:
init_x = 4

In [88]:
import random

In [89]:
values = [init_x, init_x]

In [90]:
N = 1000

In [91]:
b0 = 0.8

In [92]:
b1 = -0.4

In [93]:
noise = dnorm(0,0.1,N)

In [96]:
for i in range(N):
    new_X = values[-1] * b0 + values[-2] * b1 + noise[i]
    values.append(new_X)

This data has an AR(2) structure (two lags) with parameters 0..8 and -0.4. When we fit an AR model, we may not know the number of lagged terms to include, so we can fit the model with some larger number of lags:


In [97]:
MAXLAGS = 5

In [98]:
model = sm.tsa.AR(values)

C:\Users\Predator\Anaconda3\envs\data_analysis\lib\site-packages\statsmodels\tsa\ar_model.py:791: FutureWarning: 
statsmodels.tsa.AR has been deprecated in favor of statsmodels.tsa.AutoReg and
statsmodels.tsa.SARIMAX.

AutoReg adds the ability to specify exogenous variables, include time trends,
and add seasonal dummies. The AutoReg API differs from AR since the model is
treated as immutable, and so the entire specification including the lag
length must be specified when creating the model. This change is too
substantial to incorporate into the existing AR api. The function
ar_select_order performs lag length selection for AutoReg models.

AutoReg only estimates parameters using conditional MLE (OLS). Use SARIMAX to
estimate ARX and related models using full MLE via the Kalman Filter.

To silence this warning and continue using AR until it is removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.ar_model.AR', FutureWarning)

  warnings.warn(AR_DEPRECATION_WA

In [99]:
results = model.fit(MAXLAGS)

The estimated parameters in the results have the intercept first and the estimates for the first two lags next:

In [100]:
results.params

array([-0.00616093,  0.78446347, -0.40847891, -0.01364148,  0.01496872,
        0.01429462])

<h3>Introduction to scikit-learn</h3>

<b>scikit-learn</b> is one of the most widely used and trusted general-purpose Python machine learning toolkits. It contains a broad selection of standard supervised and unsupervised machine learning methods with tools for model selection and evaluation, data transformation, data loading, and model persistence. These models can be used for classification, clustering, prediction, and other common tasks.

scikit-learn does not have deep pandas integration, though there are some add-on third-party packages that are still in development. pandas can be very useful for massaging datasets prior to model fitting, though.

As an example, we will use a <b>now-classic dataset from a Kaggle competition</b> about passenger survival rates on the Titanic, which sank in 1912. We load the test and training dataset using pandas:

In [101]:
train = pd.read_csv('pydata-book-2nd-edition/datasets/titanic/train.csv')

In [102]:
test = pd.read_csv('pydata-book-2nd-edition/datasets/titanic/test.csv')

In [103]:
train[:4]

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S


Libraries like statsmodels and scikit-learn generally cannot be fed missing data, so we look at the columns to see if there are any that contain missing data:

In [105]:
train.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

In [106]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In statistics and machine learning examples like this one, a typical task is to predict whether a passenger would survive based on features in the data. A modle is fitted on a training dataset and then evaluated on an out-of-sample testing dataset.

I would like to use <b>Age</b> as a predicator, but it has missing data. There are a number of ways to do missing data <b>imputation</b>, but I will do a simple one and use the median of the training dataset to fill the nulls in both tables:

In [108]:
impute_value = train['Age'].median()

In [109]:
train['Age'] = train['Age'].fillna(impute_value)

In [110]:
test['Age'] = test['Age'].fillna(impute_value)

Now, we need to specify our modles. I add a column <b>IsFemale</b> as an encoded version of the 'Sex' column:

In [111]:
train['IsFemale'] = (train['Sex'] == 'female').astype(int)

In [113]:
test['IsFemale'] = (test['Sex'] == 'female').astype(int)

Then we decide on some model variables and create Numpy arrays:

In [114]:
predictors = ['Pclass', 'IsFemale', 'Age']

In [115]:
X_train = train[predictors].values

In [130]:
X_test = test[predictors].values

In [131]:
y_train = train['Survived'].values

In [132]:
X_train[:5]

array([[ 3.,  0., 22.],
       [ 1.,  1., 38.],
       [ 3.,  1., 26.],
       [ 1.,  1., 35.],
       [ 3.,  0., 35.]])

In [133]:
y_train[:5]

array([0, 1, 1, 1, 0], dtype=int64)

I make no claims that this is a good model nor that these features are engineered properly. We use the LogisticRegression model form scikit-learn to create a model instance:

In [134]:
from sklearn.linear_model import LogisticRegression

In [135]:
model = LogisticRegression()

Similar to statsmodels, we can fit this model to the training data using the model's fit method:

In [136]:
model.fit(X_train, y_train)

LogisticRegression()

Now, we can form predictions for the test dataset using model.predict:

In [137]:
y_predict = model.predict(X_test)

In [138]:
y_predict[:10]

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0], dtype=int64)

If we had the tur values for the test dataset, we could compute an accurace percentage or some other error metric: